# Baseline with mse loss with wide cnn modified + adam
#### Convert histone mark signals and use deep CNN for training.

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [1]:
%env CUDA_VISIBLE_DEVICES=6,7
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=6,7


Using TensorFlow backend.


In [2]:
window_size = 2001
process_all = False
sample_num = 1000

In [3]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [4]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [5]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [6]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [7]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [8]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 2001
[[123411027, 123413028], [123410941, 123412942], [131907564, 131909565]]


In [9]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 2001, 5)


In [10]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [11]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 2001, 5)


In [12]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 2001)


In [13]:
# replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 2001)


In [14]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 2001, 1)


In [15]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.01014  0.01014  0.01014  0.02435  0.02435]


In [16]:
from keras.layers import Input, Dense, Conv1D, Dropout
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

In [17]:
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = 2001
# parameters for second conv layer
output_filters = 1
output_kernel_size = 16
# parameters for training
batch_size = 128
num_epochs = 300
evaluation_freq = 10

In [18]:
'''
Baseline CNN (based on KERAS functional API)
'''
inputs = Input(shape=(window_size, 5, ))
x = Conv1D(
    filters=hidden_filters_1,
    kernel_size=hidden_kernel_size_1,
    padding='same',
    activation='relu',
    strides=1)(inputs)
x = Dropout(dropout_rate)(x)

outputs = Conv1D(
    filters=output_filters,
    kernel_size=output_kernel_size,
    padding='same',
    activation='linear',
    strides=1
    )(x)

model = Model(inputs=inputs, outputs=outputs)

In [19]:
# Print out model summary
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2001, 5)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2001, 32)          320192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2001, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2001, 1)           513       
Total params: 320,705
Trainable params: 320,705
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [21]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [22]:
model_dir = os.path.join("models", "mse_lowlr")
log_dir = os.path.join("logs", "mse_lowlr")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "mse_lowlr")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [23]:
# setting an adam optimizer with 1.0 clip norm
adam = optimizers.Adam(lr=0.0001, clipnorm=1.)

print "Compiling a model with adam optimizer"
model.compile(loss=losses.mean_squared_error,
              optimizer=adam,
              metrics=[pearson, metrics.mse, metrics.mae])

Compiling a model with adam optimizer


In [24]:
# CallBack: tensorboard with grad histogram
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1,
                          batch_size=batch_size,
                          write_grads=True)

In [25]:
# CallBack: reduce learning rate when validation loss meets plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.96,
                              patience=5,
                              min_lr=1e-7)

In [26]:
# CallBack: evaluate model for every evaulation epoch
class EvaluateModel(Callback):
    def __init__(self, X_test, y_test):
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        # evaluate the model
        if self.epochs % evaluation_freq == 0:
            scores = model.evaluate(X_test, y_test)
            print zip(model.metrics_names, scores)
            test_prediction = model.predict(X_test)
            test_spearman_sum = 0.0
            for idx, _pred in enumerate(test_prediction):
                rho, _ = spearmanr(histone_mark_day0[10000+idx], _pred)
                test_spearman_sum += rho
            print "Test spearman: {}".format(test_spearman_sum * 1.0 / 1000)

In [27]:
# CallBack: store bigwig file for the best model
class SaveBigwig(Callback):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.best_val_loss = float('Inf')
        self.best_epoch = -1
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0
    
    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        cur_val_loss = logs['val_loss']
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.best_epoch = self.epochs
            prediction = model.predict(X_train).reshape(7500, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[:7500], os.path.join(srv_dir, 'train'))
            prediction = model.predict(X_val).reshape(2500, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[7500:10000], os.path.join(srv_dir, 'val'))
            prediction = model.predict(X_test).reshape(1000, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[10000:11000], os.path.join(srv_dir, 'test'))
            f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
            f.write(str(self.epochs) + "  " + str(logs))
            f.close()

In [28]:
# CallBack: Save model checkpoint based on validation loss
checkpointer = ModelCheckpoint(os.path.join(model_dir, "best_model.h5"),
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True,
                               mode='min')

In [29]:
print "Fitting the model..."
X_train, y_train = atac_seq_day0[:7500], histone_mark_day0[:7500]
X_val, y_val = atac_seq_day0[7500:10000], histone_mark_day0[7500:10000]
X_test, y_test = atac_seq_day0[10000:11000], histone_mark_day0[10000:11000]

model.fit(atac_seq_day0[:10000],
          histone_mark_day0[:10000],
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.25,   
          #shuffle=True,
          callbacks=[tensorboard,
                     reduce_lr,
                     EvaluateModel(X_test, y_test),
                     checkpointer,
                     SaveBigwig(X_train, y_train, X_val, y_val, X_test, y_test)])

Fitting the model...
Train on 7500 samples, validate on 2500 samples
Epoch 1/300
7424/7500 [============================>.] - ETA: 0s - loss: 434.6080 - pearson: 0.0894 - mean_squared_error: 434.6080 - mean_absolute_error: 7.8092
Epoch 00001: val_loss improved from inf to 171.37028, saving model to models/mse_lowlr/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
7500/7500 [==============================] - 151s 20ms/step - loss: 433.7566 - pearson: 0.0894 - mean_squared_error: 433.7566 - mean_absolute_error: 7.8119 - val_loss: 171.3703 - val_pearson: 0.1210 - val_mean_squ

Epoch 10/300
1000/1000 [==============================] - 1s 636us/steps: 370.5771 - pearson: 0.1924 - mean_squared_error: 370.5771 - mean_
[('loss', 198.6124132232666), ('pearson', 0.27346986275911334), ('mean_squared_error', 198.61241351318358), ('mean_absolute_error', 5.1652755222320561)]
Test spearman: 0.338485054984

Epoch 00010: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 372.8790 - pearson: 0.1924 - mean_squared_error: 372.8790 - mean_absolute_error: 7.9192 - val_loss: 160.9130 - val_pearson: 0.1605 - val_mean_squared_error: 160.9130 - val_mean_absolute_error: 4.1118
Epoch 11/300
7424/7500 [============================>.] - ETA: 0s - loss: 371.6963 - pearson: 0.1947 - mean_squared_error: 371.6963 - mean_absolute_error: 7.8779
Epoch 00011: val_loss improved from 160.44870 to 160.18498, saving model to models/mse_lowlr/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/train.bw
W

Test spearman: 0.333710609383

Epoch 00020: val_loss improved from 159.28899 to 158.61994, saving model to models/mse_lowlr/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
7500/7500 [==============================] - 156s 21ms/step - loss: 367.7121 - pearson: 0.2028 - mean_squared_error: 367.7121 - mean_absolute_error: 7.9262 - val_loss: 158.6199 - val_pearson: 0.1671 - val_mean_squared_error: 158.6199 - val_mean_absolute_error: 4.4551
Epoch 21/300
7424/7500 [============================>.] - ETA: 0s - loss: 367.2261 - pearson: 0.2016 - mean_squared_error: 367.2261 - mean

Epoch 33/300
7424/7500 [============================>.] - ETA: 0s - loss: 357.8358 - pearson: 0.2230 - mean_squared_error: 357.8358 - mean_absolute_error: 7.8230
Epoch 00033: val_loss did not improve
7500/7500 [==============================] - 19s 3ms/step - loss: 361.0989 - pearson: 0.2225 - mean_squared_error: 361.0989 - mean_absolute_error: 7.8491 - val_loss: 158.8817 - val_pearson: 0.1728 - val_mean_squared_error: 158.8817 - val_mean_absolute_error: 4.0836
Epoch 34/300
7424/7500 [============================>.] - ETA: 0s - loss: 363.6338 - pearson: 0.2181 - mean_squared_error: 363.6338 - mean_absolute_error: 7.9422
Epoch 00034: val_loss improved from 157.65052 to 157.29764, saving model to models/mse_lowlr/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/val.bw
Wrote bedgra

Epoch 47/300
7424/7500 [============================>.] - ETA: 0s - loss: 356.9293 - pearson: 0.2268 - mean_squared_error: 356.9293 - mean_absolute_error: 7.9903
Epoch 00047: val_loss improved from 156.97553 to 156.94171, saving model to models/mse_lowlr/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_lowlr/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
7500/7500 [==============================] - 155s 21ms/step - loss: 356.5430 - pearson: 0.2273 - mean_squared_error: 356.5430 - mean_absolute_error: 7.9836 - val_loss: 156.9417 - val_pearson: 0.1761 - val_mean_squared_error: 156.9417 - val_mean_absolute_error: 4.5413
Epoch 4

...

Epoch 231/300
7424/7500 [============================>.] - ETA: 0s - loss: 333.7263 - pearson: 0.2744 - mean_squared_error: 333.7263 - mean_absolute_error: 7.9349
Epoch 00231: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 333.1994 - pearson: 0.2759 - mean_squared_error: 333.1994 - mean_absolute_error: 7.9411 - val_loss: 155.3981 - val_pearson: 0.1843 - val_mean_squared_error: 155.3981 - val_mean_absolute_error: 4.5131
Epoch 232/300
7424/7500 [============================>.] - ETA: 0s - loss: 331.3812 - pearson: 0.2702 - mean_squared_error: 331.3812 - mean_absolute_error: 7.8668
Epoch 00232: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 332.4463 - pearson: 0.2711 - mean_squared_error: 332.4463 - mean_absolute_error: 7.8769 - val_loss: 155.4251 - val_pearson: 0.1834 - val_mean_squared_error: 155.4251 - val_mean_absolute_error: 4.4588
Epoch 233/300
7424/7500 [============================>.] - ETA: 0s

7424/7500 [============================>.] - ETA: 0s - loss: 328.1490 - pearson: 0.2720 - mean_squared_error: 328.1490 - mean_absolute_error: 7.8791
Epoch 00247: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 332.2066 - pearson: 0.2745 - mean_squared_error: 332.2066 - mean_absolute_error: 7.9044 - val_loss: 155.2839 - val_pearson: 0.1840 - val_mean_squared_error: 155.2838 - val_mean_absolute_error: 4.4290
Epoch 248/300
7424/7500 [============================>.] - ETA: 0s - loss: 331.7596 - pearson: 0.2776 - mean_squared_error: 331.7596 - mean_absolute_error: 7.8429
Epoch 00248: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 331.7617 - pearson: 0.2764 - mean_squared_error: 331.7617 - mean_absolute_error: 7.8513 - val_loss: 155.6287 - val_pearson: 0.1825 - val_mean_squared_error: 155.6287 - val_mean_absolute_error: 4.4265
Epoch 249/300
7424/7500 [============================>.] - ETA: 0s - loss: 333.7

Epoch 263/300
7424/7500 [============================>.] - ETA: 0s - loss: 327.8269 - pearson: 0.2776 - mean_squared_error: 327.8269 - mean_absolute_error: 7.8208
Epoch 00263: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 330.9399 - pearson: 0.2779 - mean_squared_error: 330.9399 - mean_absolute_error: 7.8509 - val_loss: 155.3654 - val_pearson: 0.1843 - val_mean_squared_error: 155.3655 - val_mean_absolute_error: 4.5120
Epoch 264/300
7424/7500 [============================>.] - ETA: 0s - loss: 330.5586 - pearson: 0.2778 - mean_squared_error: 330.5586 - mean_absolute_error: 7.8675
Epoch 00264: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 332.3403 - pearson: 0.2773 - mean_squared_error: 332.3404 - mean_absolute_error: 7.8938 - val_loss: 156.1816 - val_pearson: 0.1862 - val_mean_squared_error: 156.1816 - val_mean_absolute_error: 4.7464
Epoch 265/300
7424/7500 [============================>.] - ETA: 0s

7424/7500 [============================>.] - ETA: 0s - loss: 331.0181 - pearson: 0.2730 - mean_squared_error: 331.0181 - mean_absolute_error: 7.8714
Epoch 00279: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 330.8194 - pearson: 0.2743 - mean_squared_error: 330.8194 - mean_absolute_error: 7.8676 - val_loss: 155.2844 - val_pearson: 0.1853 - val_mean_squared_error: 155.2844 - val_mean_absolute_error: 4.5760
Epoch 280/300
1000/1000 [==============================] - 1s 646us/steps: 331.6255 - pearson: 0.2807 - mean_squared_error: 331.6255 - mean_
[('loss', 188.82386627197266), ('pearson', 0.28587644004821777), ('mean_squared_error', 188.82386054992676), ('mean_absolute_error', 5.3858060531616214)]
Test spearman: 0.302499679886

Epoch 00280: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 330.9773 - pearson: 0.2811 - mean_squared_error: 330.9773 - mean_absolute_error: 7.8986 - val_loss: 155.2138 - val_pe

Epoch 295/300
7424/7500 [============================>.] - ETA: 0s - loss: 331.1387 - pearson: 0.2837 - mean_squared_error: 331.1387 - mean_absolute_error: 7.9165
Epoch 00295: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 330.2792 - pearson: 0.2834 - mean_squared_error: 330.2792 - mean_absolute_error: 7.9128 - val_loss: 155.7295 - val_pearson: 0.1859 - val_mean_squared_error: 155.7295 - val_mean_absolute_error: 4.6799
Epoch 296/300
7424/7500 [============================>.] - ETA: 0s - loss: 332.2216 - pearson: 0.2797 - mean_squared_error: 332.2216 - mean_absolute_error: 7.8754
Epoch 00296: val_loss did not improve
7500/7500 [==============================] - 18s 2ms/step - loss: 330.5610 - pearson: 0.2806 - mean_squared_error: 330.5610 - mean_absolute_error: 7.8593 - val_loss: 155.1028 - val_pearson: 0.1848 - val_mean_squared_error: 155.1028 - val_mean_absolute_error: 4.4930
Epoch 297/300
7424/7500 [============================>.] - ETA: 0s